In [537]:
from environment import utils, Environment
from importlib import reload
import matplotlib.pyplot as plt


from absl import flags
FLAGS = flags.FLAGS

import numpy as np

if not FLAGS.find_module_defining_flag('vega_obs', default=None): 
    flags.DEFINE_boolean('vega_obs', True, 'Include portfolio vega and hedging option vega in state variables (Default False)')
# set vega obs to True to include vega in state variables
FLAGS['vega_obs'].value = True
FLAGS.mark_as_parsed()



In [538]:
np.set_printoptions(precision=10, suppress=True)
np.set_printoptions(threshold=np.inf, linewidth=np.inf)

In [539]:
reload(utils)
reload(Environment)
util = utils.Utils()

env = Environment.TradingEnv(utils=util)
%load_ext autoreload
%autoreload 2

env.reset()

utils initiated with spread=0, poisson_rate=1, n_episodes=1000

Memory usage before lmm: 433.07 MB
t_max set to 7
!!!! CONTRACT SIZE IS  100.0
Memory usage after: 433.89 MB

Memory usage after gc: 433.89 MB

initializing classes
done initializing classes
Main portfolio initialized with kernel beta of  1.0  using triangle kernel
REMEMBER TO ADD BACK THE SENSITIVITY FOR LIAB AND HEDGE WHEN USING THE RBF KERNEL
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/hedge/venv/lib/python3.9/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


array([  0.          ,   1.5908656   ,   1.9754823   ,   0.          , 345.13095     ,   0.          ,   0.0002432485], dtype=float32)

In [540]:

print(env.t)
env.step([0,1])[2]
env.portfolio.get_gamma(env.t)
env.portfolio.get_gamma_local_hed(env.t)
env.portfolio.get_vega

0
cost of swaption and position size -0.0 -0.0


<bound method MainPortfolio.get_vega of <environment.Trading.MainPortfolio object at 0x7f16c918b280>>

In [547]:
env.portfolio.get_delta_vec(env.t).shape[0]-env.portfolio.get_delta_vec(env.t).shape[0]/2

52.0

In [541]:
from functools import partial
def get_instant_vol_func(tau_i , a,b,c,d):

    tau = np.maximum(tau_i, 0)
    instantaneous_vol = (a + b * tau) * np.exp(-c * tau) + d
    return instantaneous_vol

def doust_corr(tau_i,tau_j, eta, beta):
    return eta + (1-eta) * np.exp(-beta*np.abs(tau_i-tau_j))

def trainable_kernel(tau_i, tau_j, a, b, c, d, eta, beta):
    return doust_corr(eta, beta, tau_i,tau_j) * get_instant_vol_func(tau_j , a,b,c,d) / get_instant_vol_func(tau_i , a,b,c,d)
    


low_bounds = {'a': 1, 'b': 1, 'c': 1, 'd': 0.001}

ts = np.linspace(0,3,105+52) # 3 yers
print(ts.shape)
vols = get_instant_vol_func(ts,low_bounds.values())
delta_kernel =  vols[:,None]/vols[52] # normalized to 1y mark
 
plt.plot(ts,delta_kernel)
plt.show()

(157,)


TypeError: get_instant_vol_func() missing 3 required positional arguments: 'b', 'c', and 'd'

In [561]:
ts = np.arange(0,3,1/52) # 3 yers
ts[104]

2.0

In [562]:
import sonnet as snt
import tensorflow as tf
import tensorflow_probability as tfp

# Use short alias for bijectors
tfb = tfp.bijectors

class OneYearKernel(snt.Module):
    """
    Trainable 1D kernel anchored at a fixed index (e.g., 52 for 1y), over
    an input tenor vector of length T.  
    Computes
        vols_j = f(tau_j)
        vol_ref = vols_j[anchor_idx]
        index_dist = |j - anchor_idx|
        corr_j = eta + (1-eta)*exp(-beta*index_dist)
        kernel_j = corr_j * (vols_j/vol_ref)

    All parameters (a,b,c,d,eta,beta) are constrained to (0,1).
    """
    def __init__(self,
                 init_params=None,
                 anchor_index=52,
                 name=None):
        super().__init__(name=name)
        # Default initial values in (0,1) space: a,b,c,d,eta,beta
        defaults = init_params or {
            'a': 0.5,
            'b': 0.5,
            'c': 0.5,
            'd': 0.001,
            'eta': 0.5,
            'beta': 0.5,
        }
        sigmoid = tfb.Sigmoid()
        # Rebonato volatility function parameters
        self.a = tfp.util.TransformedVariable(defaults['a'], sigmoid,
                                              dtype=tf.float32, name='a')
        self.b = tfp.util.TransformedVariable(defaults['b'], sigmoid,
                                              dtype=tf.float32, name='b')
        self.c = tfp.util.TransformedVariable(defaults['c'], sigmoid,
                                              dtype=tf.float32, name='c')
        self.d = tfp.util.TransformedVariable(defaults['d'], sigmoid,
                                              dtype=tf.float32, name='d')
        # DOUST-style correlation parameters
        self.eta = tfp.util.TransformedVariable(defaults['eta'], sigmoid,
                                                dtype=tf.float32, name='eta')
        self.beta = tfp.util.TransformedVariable(defaults['beta'], sigmoid,
                                                 dtype=tf.float32, name='beta')
        # Anchor index in the tenor vector (e.g. 52 for 1y)
        self.anchor_index = anchor_index

    def get_instant_vol(self, tau):
        """
        Instantaneous vol function f(tau) = (a + b*tau)*exp(-c*tau) + d
        """
        tau = tf.maximum(tau, 0.0)
        return (self.a + self.b * tau) * tf.exp(-self.c * tau) + self.d

    def __call__(self, tau_j):
        """
        Args:
          tau_j: 1D tensor of shape [T] containing tenor points (years).
        Returns:
          kernel: 1D tensor [T] of normalized weights.
        """
        # Compute instantaneous vols for each tenor
        vols_j = self.get_instant_vol(tau_j)  # [T]
        # Use the anchor tenor's vol for normalization
        vol_ref = vols_j[self.anchor_index]   # scalar

        # Build index distance vector
        T = tf.shape(tau_j)[0]
        idx = tf.cast(tf.range(T), tf.float32)
        anchor_idx = tf.cast(self.anchor_index, tf.float32)
        index_dist = tf.abs(idx - anchor_idx)  # [T]

        # DOUST-style correlation based on index distance
        corr_j = self.eta + (1. - self.eta) * tf.exp(-self.beta * index_dist)

        # Final kernel weights
        kernel = corr_j * (vols_j / vol_ref)
        return kernel

/home/hedge/venv/lib/python3.9/site-packages/sonnet/src/types.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  BoolLike = Union[bool, np.bool, TensorLike]
